In [1]:
##notebook p/ extrair recomends do modlo sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
##melhor rodar em noteboook pq kernel guarda em cache os embeddings
##para implemenatr em prod, melhpr solucao é guardar vetores de embds gerados em um chromadb com
##atualizacao incremental sempre que novos itens entrarem na base

from BERT_recomend_construct import Recommender
import os
from dotenv import load_dotenv
import pandas as pd


load_dotenv()

model = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
##Para o DB de cursos:
emb_cols = ['apresentacao', 'conteudo_programatico']

##Para o DB de livros:
#emb_cols = ['description']

recomender = Recommender(
    DB_USER = os.getenv("DB_USER"),
    DB_PASS=os.getenv("DB_PASS"),
    DB_NAME=os.getenv("DB_NAME"),
    DB_HOST=os.getenv("DB_HOST"),
    model_name=model,
    emb_cols=emb_cols,
    # id_col='isbn',
    # item_name_col='title'
    id_col= 'id_curso',
    item_name_col = 'nome_curso'
)

##carrego a tabela do DB (aqui colocar livros ou cursos)
recomender.carrega_dados('cursos')
##aplico limpeza simples aos dados
recomender.limpa_dados()
##extraio os embeddings
embeddings = recomender.embeddings_extract()

# recomendacoes:(homem mais rico babi)
#recomendacoes = recomender.recomendar_itens("9788500003042", 10)
#recomendacoes = recomender.recomendar_itens(113, 3)
#recomendacoes


/Users/pedroh.mello/.pyenv/versions/3.12.3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Dados textuais carregados com sucesso 

Dados limpos, padronizados e texto compilado em uma única coluna.



Processando Embeddings: 100%|██████████| 13/13 [00:04<00:00,  2.92it/s]

Vetores de representação textual gerados com sucesso pelo PyTorch!




In [5]:
from tqdm import tqdm

def generate_all_recommendations(recomender, top_n=3):
    """
    Generate recommendations for all courses in the loaded dataframe.
    Returns a DataFrame with course names instead of IDs in the first column.
    """
    # Get all course IDs and names from the dataframe
    all_courses = recomender.df_text[[recomender.id_col, 'nome_original']].astype({recomender.id_col: str})
    
    recommendations_list = []
    
    for _, row in tqdm(all_courses.iterrows(), total=len(all_courses), desc="Processing Courses"):
        course_id = row[recomender.id_col]
        base_course_name = row['nome_original']
        
        try:
            # Get recommendations for current course
            result = recomender.recomendar_itens(course_id, top_n)
            
            if result is None or result.empty:
                continue
                
            # Create a dictionary for the current course
            course_dict = {
                'ultimo_curso_usuario': base_course_name,
            }
            
            # Extract recommendations and similarity scores
            for i in range(top_n):
                try:
                    course_dict[f'curso_recomendado_{i+1}'] = result.iloc[i]['Item Recomendado']
                    course_dict[f'similaridade_coss_curso{i+1}'] = result.iloc[i]['Similaridade Cosseno']
                except (IndexError, KeyError):
                    course_dict[f'curso_recomendado_{i+1}'] = None
                    course_dict[f'similaridade_coss_curso{i+1}'] = None
            
            recommendations_list.append(course_dict)
            
        except Exception as e:
            print(f"Error processing course {base_course_name} ({course_id}): {str(e)}")
            continue
    
    # Create DataFrame and format columns
    column_order = ['ultimo_curso_usuario']
    for i in range(1, top_n+1):
        column_order.extend([
            f'curso_recomendado_{i}',
            f'similaridade_coss_curso{i}'
        ])
    
    return pd.DataFrame(recommendations_list, columns=column_order)

# Usage remains the same:
final_recommendations = generate_all_recommendations(recomender, top_n=3)
final_recommendations.head()

Processing Courses:  29%|██▊       | 232/807 [00:00<00:00, 1168.66it/s]

As recomendações mais similares ao item 'Parcerias Público-Privadas E Concessões' são:

As recomendações mais similares ao item 'Treinamento Censo Superior' são:

As recomendações mais similares ao item 'Uso Responsável Das Tics' são:

As recomendações mais similares ao item 'Type I Environmental Labeling Course: Sustainability And Competitiveness For Brazilian Products And Services' são:

As recomendações mais similares ao item 'Educação Patrimonial E Turismo Cultural' são:

As recomendações mais similares ao item 'Educação Museal Ii - Histórico E Práticas' são:

As recomendações mais similares ao item 'Noções De Ciência Política Aplicada À Gestão De Recursos Hídricos' são:

As recomendações mais similares ao item 'S2Id - M3 - Usuário Municipal - Solicitação De Recursos Para Obras De Reconstrução' são:

As recomendações mais similares ao item 'Scrum In The Context Of The Public Sector' são:

As recomendações mais similares ao item 'Trilho 2 - Atuando Em Rede Trilha - Coordenação E Col

Processing Courses:  58%|█████▊    | 468/807 [00:00<00:00, 1170.78it/s]

As recomendações mais similares ao item 'S2Id - M5 – Plantonista (Todos) - Monitoramento' são:

As recomendações mais similares ao item 'Termo De Execução Descentralizada: Visão Geral E Atos Preparatórios' são:

As recomendações mais similares ao item 'Análise De Impacto Regulatório: Conceitos Fundamentais' são:

As recomendações mais similares ao item 'Promoção E Defesa Dos Direitos Das Pessoas Lgbtqia+' são:

As recomendações mais similares ao item 'Gerenciamento De Continuidade De Negócios Na Administração Pública' são:

As recomendações mais similares ao item 'Trilho 1 - Tomando Decisões Com Base Na Ética Trilha - Ética E Integridade Pública' são:

As recomendações mais similares ao item 'Segurança Da Informação No Contexto Da Transformação Digital' são:

As recomendações mais similares ao item 'A Construção Dos Livros Didáticos Do Pnld' são:

As recomendações mais similares ao item 'População Em Situação De Rua E O Modelo Moradia Primeiro (Housing First)' são:

As recomendações ma

Processing Courses:  88%|████████▊ | 710/807 [00:00<00:00, 1191.18it/s]

As recomendações mais similares ao item 'Sistemas De Monitoramento E Alerta Como Suporte À Gestão Local De Riscos E Desastres' são:

As recomendações mais similares ao item 'Biossegurança Em Laboratórios De Ensino E Pesquisa' são:

As recomendações mais similares ao item 'Introdução À Economia Do Setor Público' são:

As recomendações mais similares ao item 'Instrumentos De Desenvolvimento Urbano Sustentável - Informações' são:

As recomendações mais similares ao item 'Fundamentos De Governança E Boas Práticas Da Regulação' são:

As recomendações mais similares ao item 'Prevenção À Lavagem De Dinheiro E Ao Financiamento Do Terrorismo' são:

As recomendações mais similares ao item 'Regulação De Sistemas De Saúde Do Sus' são:

As recomendações mais similares ao item 'Design Thinking Aplicado A Bibliotecas' são:

As recomendações mais similares ao item 'Transferegov.Br: Cadastro De Entes, Instituições E Usuários' são:

As recomendações mais similares ao item 'Gestão De Projetos Educacionai

Processing Courses: 100%|██████████| 807/807 [00:00<00:00, 1178.95it/s]

As recomendações mais similares ao item 'Técnica Legislativa Aplicada À Receita Federal Do Brasil' são:

As recomendações mais similares ao item 'Contencioso Aduaneiro E Tributário No Âmbito Da Receita Federal' são:

As recomendações mais similares ao item 'Atendimento Pré-Hospitalar Em Acidentes Com Armas De Fogo Aplicado À Receita Federal' são:

As recomendações mais similares ao item 'Planejamento Sucessório E Tributário Aplicado À Receita Federal' são:

As recomendações mais similares ao item 'Emag Conteudista' são:

As recomendações mais similares ao item 'Documentos Fiscais Aplicados À Vigilância E Repressão De Ações Da Receita Federal' são:

As recomendações mais similares ao item 'Educação Patrimonial E Educação Museal: Relações E Interconexões Possíveis' são:

As recomendações mais similares ao item 'Mercado De Câmbio Aplicado À Receita Federal' são:

As recomendações mais similares ao item 'Conceitos De Fiscalização Aduaneira' são:

As recomendações mais similares ao item 'Su

,ultimo_curso_usuario,curso_recomendado_1,similaridade_coss_curso1,curso_recomendado_2,similaridade_coss_curso2,curso_recomendado_3,similaridade_coss_curso3
0,Parcerias Público-Privadas e Concessões,Celebração de Parcerias no Setor Público,0.765623,Gestão de Convênios para Concedentes,0.747030,Planejamento da Contratação de Soluções de TIC,0.716497
1,Treinamento Censo Superior,Formação de Professores do Programa Aprender Valor,0.801590,Formação de Facilitadores de Aprendizagem,0.793030,Formação de Gestores Escolares do Programa Aprender Valor,0.733472
2,Uso responsável das TICs,Internet das coisas (IoT) aplicada para resolução de desafios,0.680690,Trilho 3 - Integrando comunicação e tecnologias digitais Trilha - Mentalidade Digital,0.644404,Garantia de Direitos e Atenção à Pessoa Idosa,0.611961
3,Type I Environmental Labeling course: Sustainability and Competitiveness for Brazilian Products and Services,Etiquetado Ambiental Tipo I: sostenibilidad y competitividad para productos y servicios brasileños,0.912234,Rotulagem Ambiental Tipo I: Sustentabilidade e Competitividade para Produtos e Serviços Brasileiros,0.874780,Sustentabilidade na Administração Pública,0.621054
4,Educação Patrimonial e Turismo Cultural,Educação Patrimonial e Patrimônio Material,0.782034,Educação Patrimonial e Inventários Participativos,0.771405,Educação Patrimonial e Diversidade,0.758589


In [6]:
final_recommendations.to_excel("/Users/pedroh.mello/Desktop/recomends_matriz_EVG.xlsx", index=False)

In [3]:
reco = recomender.recomendar_itens(930, 3)
reco.to_csv("/Users/pedroh.mello/Desktop/MESTRADO_MCDE/SEMINARIO/PROJETO_CONCLUSAO_MESTRADO/data/tbls_recomends_resultado/sbert_curso_930.csv")

As recomendações mais similares ao item 'Estatística Para Análise De Dados Na Administração Pública' são:



In [5]:
reco = recomender.recomendar_itens(889, 3)

reco.to_csv("/Users/pedroh.mello/Desktop/MESTRADO_MCDE/SEMINARIO/PROJETO_CONCLUSAO_MESTRADO/data/tbls_recomends_resultado/sbert_curso_889.csv")

As recomendações mais similares ao item 'Conceitos Essenciais Sobre Patologias Em Estruturas De Concreto' são:



In [ ]:
##TALVEZ
reco = recomender.recomendar_itens(679, 3)

reco

As recomendações mais similares ao item 'Introdução À Gestão De Processos' são:



,Item Recomendado,Similaridade Cosseno
1,Gerenciamento de Projetos na Prática,0.710428
2,Análise e Melhoria de Processos,0.663576
3,Gestão de projetos,0.643690


In [3]:
recomendacoes2 = recomender.recomendar_itens(889, 3)

recomendacoes2

As recomendações mais similares ao item 'Conceitos Essenciais Sobre Patologias Em Estruturas De Concreto' são:



,Item Recomendado,Sim_Coss
1,Reconhecimento de Riscos Químicos nos Ambiente...,0.676298
2,Relatório de Segurança de Barragens: o que é e...,0.607058
3,Conceitos Básicos de Hidrologia e Drenagem par...,0.600677


In [ ]:
# recomendacoes:
recomendacoes2 = recomender.recomendar_itens("9788501038012", 10)
print(recomendacoes2)

ID 9788501038012 não encontrado na base de dados.
None


In [6]:
recomendacoes3 = recomender.recomendar_itens(390, 3)

recomendacoes3

As recomendações mais similares ao item 'Etapas E Procedimentos Do Pnld' são:



,Item Recomendado,Sim_Coss
1,A importância do PNLD,0.840634
2,PDCA Aplicado à Educação Básica,0.803201
3,Desenho Instrucional para Soluções de Capacita...,0.769333


In [29]:
recomendacoes4 = recomender.recomendar_itens(92, 3)

recomendacoes4.to_csv("/Users/pedroh.mello/Desktop/MESTRADO_MCDE/SEMINARIO/PROJETO_CONCLUSAO_MESTRADO/data/tbls_recomends_resultado/sbert_curso_92.csv")

As recomendações mais similares ao item 'Microeconomia' são:



In [4]:
recomendacoes5 = recomender.recomendar_itens(629, 3)

recomendacoes5

recomendacoes5.to_csv("/Users/pedroh.mello/Desktop/MESTRADO_MCDE/SEMINARIO/PROJETO_CONCLUSAO_MESTRADO/data/tbls_recomends_resultado/sbert_curso_629.csv")

As recomendações mais similares ao item 'Aprendendo Com Python' são:



In [11]:
recomendacoes6 = recomender.recomendar_itens(1130, 3)

recomendacoes6

As recomendações mais similares ao item 'Como Trabalhar Com Computadores E Dispositivos Móveis' são:



,Item Recomendado,Sim_Coss
1,Como trabalhar e colaborar On-line,0.714734
2,Visão Geral da IA no Azure,0.701771
3,Otimização de trabalho com o Microsoft 365,0.689473


In [12]:
recomendacoes7 = recomender.recomendar_itens(800, 3)

recomendacoes7

As recomendações mais similares ao item 'Big Data Em Apoio À Tomada De Decisão' são:



,Item Recomendado,Sim_Coss
1,Análise de dados como suporte à tomada de decisão,0.759159
2,Introdução à Ciência de Dados - Tratamento de ...,0.695739
3,Governança de Dados,0.682540


In [13]:
recomendacoes8 = recomender.recomendar_itens(486, 3)

recomendacoes8

As recomendações mais similares ao item 'Nova Lei De Licitações: Visão Geral' são:



,Item Recomendado,Sim_Coss
1,Nova Lei de Licitações: Gestão Contratual,0.858196
2,Nova Lei de Licitações: Sanções ao fornecedor,0.813717
3,"Licitação por concorrência, concurso, leilão e...",0.798455


In [14]:
recomendacoes8 = recomender.recomendar_itens(410, 3)

recomendacoes8

As recomendações mais similares ao item 'Estudo Técnico Preliminar Avançado Para Contratação De Soluções De Tic' são:



,Item Recomendado,Sim_Coss
1,Termo de Referência ou Projeto Básico para a C...,0.813183
2,Contratações de Encomendas Tecnológicas,0.767884
3,Planejamento da Contratação de Soluções de TIC,0.723736


# Recomendações para a base de livros

In [1]:
##notebook p/ extrair recomends do modlo sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
##melhor rodar em noteboook pq kernel guarda em cache os embeddings
##para implemenatr em prod, melhpr solucao é guardar vetores de embds gerados em um chromadb com
##atualizacao incremental sempre que novos itens entrarem na base

from BERT_recomend_construct import Recommender
import os
from dotenv import load_dotenv
import pandas as pd


load_dotenv()

model = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
##Para o DB de cursos:
#emb_cols = ['apresentacao', 'conteudo_programatico']

##Para o DB de livros:
emb_cols = ['description']

recomender = Recommender(
    DB_USER = os.getenv("DB_USER"),
    DB_PASS=os.getenv("DB_PASS"),
    DB_NAME=os.getenv("DB_NAME"),
    DB_HOST=os.getenv("DB_HOST"),
    model_name=model,
    emb_cols=emb_cols,
    id_col='isbn',
    item_name_col='title'
    # id_col= 'id_curso',
    # item_name_col = 'nome_curso'
)

##carrego a tabela do DB (aqui colocar livros ou cursos)
recomender.carrega_dados('livros')
##aplico limpeza simples aos dados
recomender.limpa_dados()
##extraio os embeddings
embeddings = recomender.embeddings_extract()

# recomendacoes:(homem mais rico babi)
# recomendacoes = recomender.recomendar_itens(9788500003042, 10)
# #recomendacoes = recomender.recomendar_itens(113, 3)
# recomendacoes

/Users/pedroh.mello/.pyenv/versions/3.12.3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Dados textuais carregados com sucesso 

Dados limpos, padronizados e texto compilado em uma única coluna.



Processando Embeddings: 100%|██████████| 466/466 [02:30<00:00,  3.09it/s]


Vetores de representação textual gerados com sucesso pelo PyTorch!




In [8]:
recomendacoes.to_csv("/Users/pedroh.mello/Desktop/MESTRADO_MCDE/SEMINARIO/PROJETO_CONCLUSAO_MESTRADO/data/tbls_recomends_resultado/sbert_livro_9788500003042.csv")

In [2]:
##TALVEZ
recs = recomender.recomendar_itens(9788500003042,6)
recs
recs.to_csv("/Users/pedroh.mello/Desktop/MESTRADO_MCDE/SEMINARIO/PROJETO_CONCLUSAO_MESTRADO/data/tbls_recomends_resultado/sbert_livro_9788500003042.csv")

As recomendações mais similares ao item 'Homem Mais Rico Da Babilônia,O' são:



In [3]:
embeddings.shape

torch.Size([29793, 384])

In [35]:
embeddings[1]

tensor([-8.0443e-03,  1.1683e-01, -3.4714e-02,  2.4540e-01,  4.3900e-02,
         4.2208e-02,  2.4497e-01,  1.3763e-01,  6.9532e-02,  9.3704e-02,
         2.2520e-01,  1.6017e-01,  8.6985e-02,  7.5492e-02, -1.0668e-01,
         1.6597e-01, -1.1095e-01,  6.5783e-02, -3.1311e-02,  1.0069e-01,
        -1.1623e-01,  1.1052e-01,  3.0532e-01,  4.6286e-03, -3.4488e-01,
        -5.1334e-02,  1.0477e-01, -1.2785e-01, -9.9736e-02, -1.2087e-01,
         1.1447e-01, -7.9487e-02, -1.1510e-02,  1.7273e-01, -7.4933e-02,
         1.4735e-01, -5.0829e-02,  1.2077e-01, -1.2804e-01, -7.2844e-02,
        -1.3341e-01,  2.3904e-02, -1.5432e-02, -3.0863e-02,  6.0341e-02,
         1.3858e-01, -9.0612e-03, -3.0270e-02,  9.9787e-02, -1.1922e-01,
        -2.8243e-01, -3.3097e-02, -2.6117e-02, -4.7149e-02,  1.2167e-01,
         8.2846e-02,  1.8885e-01, -4.5142e-02, -6.7050e-02,  3.4540e-01,
         9.8576e-02,  2.7733e-02, -1.9730e-01, -3.5011e-03,  6.8074e-02,
        -1.5803e-01, -5.4078e-02, -6.7673e-02,  1.0

In [8]:
recomender.recomendar_itens(9788500012952,6)

As recomendações mais similares ao item 'Bíblia Do Vinho, A' são:



,Item Recomendado,Similaridade Cosseno
1,"Vinhos - Degustação, Elaboração e Serviço",0.769939
2,O Guia Essencial do Vinho,0.766739
3,Vinhos e comida,0.757899
4,Os ignorantes,0.747955
5,Uma breve história da bebedeira,0.728047
6,A alma do vinho,0.726168


In [19]:
recomender.recomendar_itens("9788501038012",10)

As recomendações mais similares ao item 'guerreiro pagão - crônicas saxônicas - vol. 7' são:

                                    Item Recomendado  Sim_Coss
0         guerra lobo - crônicas saxônicas - vol. 11  0.809031
1          trono vazio - crônicas saxônicas - vol. 8  0.791403
2       portador fogo - crônicas saxônicas - vol. 10  0.784609
3  guerreiros tempestade - crônicas saxônicas - v...  0.770763
4        guerreiro pagão (vol. 7 crônicas saxônicas)  0.748087
5           guerra lobo (vol. 11 crônicas saxônicas)  0.747763
6      cavaleiro morte - crônicas saxônicas - vol. 2  0.680716
7            trono vazio (vol. 8 crônicas saxônicas)  0.679548
8           espada reis (vol. 12 crônicas saxônicas)  0.673363
9        canção espada - crônicas saxônicas - vol. 4  0.658912


In [6]:
recomender.recomendar_itens(9788594318572,10)

As recomendações mais similares ao item 'Livro Do Desassossego' são:



,Item Recomendado,Similaridade Cosseno
1,O homem despedaçado,0.798158
2,Fragmentos sobre o que se escreve de uma psicanálise,0.791037
3,A Metapoesia Desvairada de Mário de Andrade,0.787451
4,Leituras desarquivadas,0.785937
5,Procura do romance,0.779218
6,O Livro E Seus Algozes,0.770106
7,A literatura em perigo,0.768928
8,O tempo,0.767328
9,Dentro do Dentro,0.766282
10,Livro de auras,0.760106
